Used seq2seq encoder-decoder architecture, which uses LSTM (Long Short Term Memory), encodes the input hindi sequence into a single vector. This vector is then sent into the decoder neural network, which produces the English translation sentence.

#Imports 

In [ ]:
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [ ]:
!pip install torchtext==0.6.0 --quiet
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.4 MB/s eta 0:00:00


'\n# Seeding for reproducible results everytime\nSEED = 777\n\nrandom.seed(SEED)\nnp.random.seed(SEED)\ntorch.manual_seed(SEED)\ntorch.cuda.manual_seed(SEED)\ntorch.backends.cudnn.deterministic = True'

In [ ]:
import re
from indicnlp.tokenize import indic_tokenize

In [ ]:
!python -m spacy download en_core_web_sm --quiet


2023-04-11 14:19:51.844968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
nlp_en = spacy.load("en_core_web_sm")

In [ ]:
! pip install indic-nlp-library

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 18.8 MB/s eta 0:00:00


In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 860.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not curre

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"

Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 1362 (delta 111), reused 98 (delta 93), pack-reused 1219
Receiving objects: 100% (1362/1362), 9.56 MiB | 12.04 MiB/s, done.
Resolving deltas: 100% (721/721), done.


In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [ ]:
!pip install Morfessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
INDIC_NLP_LIB_HOME=r"/content/indic_nlp_library"
INDIC_NLP_RESOURCES="/content/indic_nlp_resources"

In [ ]:
from indicnlp.tokenize import sentence_tokenize
from indicnlp.tokenize import indic_tokenize  

# DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# base directory path, CS779_NLP_COMP/1_MT
base_dir = '/content/drive/MyDrive/CS779_NLP_COMP/1_MT/'

# Access files, /input_data
folder_name = 'input_data'

# file_path
file_path = base_dir + folder_name + '/'

In [ ]:
!nvidia-smi

Tue Apr 11 14:19:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data Pre-processing (Using torchtext and Spacy)

In [ ]:
# CS779_NLP_COMP/1_MT/input_data/eng_Hindi_data_train.csv
df_train = pd.read_csv(file_path+"eng_Hindi_data_train.csv", header = None)
df_train.head()

,0,1
0,and deliver us by Thy mercy from the people of...,और अपनी रहमत से हमें इन काफ़िर लोगों (के नीचे)...
1,Transformed position of fourth point,चौथे बिन्दु का रूपांतरित स्थान
2,"Oh, woe to me; I wish I never took so - and - ...",हाए अफसोस काश मै फला शख्स को अपना दोस्त न बनाता
3,The PS file is to be translated into a PDF fil...,पीएस2पीडीएफ के इस्तेमाल से पीएस फ़ाइल को पीडीए...
4,Receiving LDAP search results...,LDAP खोज परिणाम पा रहा है...


In [ ]:
df_train.columns = ['English', 'Hindi']
df_train.head()

,English,Hindi
0,and deliver us by Thy mercy from the people of...,और अपनी रहमत से हमें इन काफ़िर लोगों (के नीचे)...
1,Transformed position of fourth point,चौथे बिन्दु का रूपांतरित स्थान
2,"Oh, woe to me; I wish I never took so - and - ...",हाए अफसोस काश मै फला शख्स को अपना दोस्त न बनाता
3,The PS file is to be translated into a PDF fil...,पीएस2पीडीएफ के इस्तेमाल से पीएस फ़ाइल को पीडीए...
4,Receiving LDAP search results...,LDAP खोज परिणाम पा रहा है...


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

## Tokenization: 


In [ ]:
# Tokenize text using IndicNLP for Hindi and Spacy for English
def preprocess_text_en(text):
    # Lowercase
    text = text.lower()
    # Clean
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    doc = nlp_en(text)
    tokens = [token.text for token in doc]
    return tokens

# using IndicNLP
def preprocess_text_hi(text):
    # Lowercase
    text = text.lower()
    tokens = []
    # Clean
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    sentences= sentence_tokenize.sentence_split(text, lang='hi')
    for t_s in sentences:
      for t in indic_tokenize.trivial_tokenize(t_s):
        tokens.append(t)
    return tokens

In [ ]:
# Test Run
sample = "This is Machine Translation model"
print(preprocess_text_en(sample))

## Train/ Valid/ Test Split: 

In [ ]:
len(df_train)

140000

In [ ]:
train = df_train[:110000]
test = df_train[120000:130000]
valid = df_train[130000:]

train.to_csv('train.csv', index = False)
test.to_csv('test.csv', index = False)
valid.to_csv('valid.csv', index = False)

## Generate vocabulary
https://torchtext.readthedocs.io/en/latest/data.html

In [ ]:
# Fields for hindi and english languages
hindi = Field(tokenize=preprocess_text_hi, lower=True, init_token="<sos>", eos_token="<eos>")
english = Field(tokenize=preprocess_text_en, lower=True, init_token="<sos>", eos_token="<eos>")

In [ ]:
# Load train, validation, test data as TabularDataset
train_data = TabularDataset(path='train.csv', format='csv', fields=[('trg', english), ('src', hindi)])
valid_data = TabularDataset(path='valid.csv', format='csv', fields=[('trg', english), ('src', hindi)])
test_data = TabularDataset(path='test.csv', format='csv', fields=[('trg', english), ('src', hindi)])

In [ ]:
# Build vocabulary
hindi.build_vocab(train_data, min_freq=2)
english.build_vocab(train_data, min_freq=2)

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Set batch size
BATCH_SIZE = 32

In [ ]:
# iterators for train, validation and test data
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, sort_within_batch=True, 
                                                                      sort_key=lambda x: len(x.src), device = device)

# 5. Encoder

EncoderLSTM(
  <br>
  (dropout): Dropout(p=0.3, inplace=False)
  <br>
  (embedding): Embedding(12271, 500)
  <br>
  (LSTM): LSTM(500, 1024, num_layers=2, dropout=0.3)
  <br>
)

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()
    # Hidden size
    self.hidden_size = hidden_size
    # Num layers 
    self.num_layers = num_layers
    # Define dropout layer
    self.dropout = nn.Dropout(p)
    self.tag = True
    # Define embedding layer
    self.embedding = nn.Embedding(input_size, embedding_size)
    # Define LSTM layer
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    # embedded input to LSTM layer
    outputs, (hidden_state, cell_state) = self.LSTM(embedding)
    return hidden_state, cell_state

In [ ]:
# hyperparameters
encoder_embedding_size = 500
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.3
encoder_input_size = len(hindi.vocab)
# Instantiate EncoderLSTM
encoder_lstm = EncoderLSTM(encoder_input_size, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

# Decoder

DecoderLSTM(
  <br>
  (dropout): Dropout(p=0.3, inplace=False)
  <br>
  (embedding): Embedding(16523, 500)
  <br>
  (LSTM): LSTM(500, 1024, num_layers=2, dropout=0.3)
  <br>
  (fc): Linear(in_features=1024, out_features=16523, bias=True)
  <br>
)

In [ ]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()
    # Hidden size
    self.hidden_size = hidden_size
    # Num layers
    self.num_layers = num_layers
    # Output size
    self.output_size = output_size
    # Dropout layer
    self.dropout = nn.Dropout(p)
    # Embedding layer
    self.embedding = nn.Embedding(input_size, embedding_size)
    # LSTM layer
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    # Fully connected layer
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x, hidden_state, cell_state):
    x = x.unsqueeze(0)
    # Embed input tensor
    embedding = self.dropout(self.embedding(x))
    # embedding tensor, previous hidden, cell states through LSTM layer
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))
    # LSTM output through fully connected layer
    predictions = self.fc(outputs)
    # Remove added dimension and return predictions
    predictions = predictions.squeeze(0)
    return predictions, hidden_state, cell_state



In [ ]:
# hyperparameters
decoder_embedding_size = 500
hidden_size = 1024
num_layers = 2
decoder_dropout = 0.3
decoder_input_size = len(english.vocab)
output_size = len(english.vocab)
# Instantiate DecoderLSTM
decoder_lstm = DecoderLSTM(decoder_input_size, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

# Seq2Seq

Seq2Seq(
  <br>
  (Encoder_LSTM): EncoderLSTM(
    <br>
    (dropout): Dropout(p=0.3, 
    <br>
    inplace=False)
    <br>
    (embedding): Embedding(12271, 500)
    <br>
    (LSTM): LSTM(500, 1024, num_layers=2, 
    <br>
    dropout=0.3)
    <br>
  )
  <br>
  (Decoder_LSTM): DecoderLSTM(
    <br>
    (dropout): Dropout(p=0.3, 
    <br>
    inplace=False)
    <br>
    (embedding): Embedding(16523, 500)
    <br>
    (LSTM): LSTM(500, 1024, num_layers=2,
    <br>
     dropout=0.3)
     <br>
    (fc): Linear(in_features=1024, 
    <br>
    out_features=16523, bias=True)
    <br>
  )
  <br>
)

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    # initialize the Encoder and Decoder LSTM
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # get batch size, target length, and target vocabulary size
    batch_size = source.shape[1]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)
    # outputs tensor with zeros
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
    hidden_state, cell_state = self.Encoder_LSTM(source)
    # initialize first input with the first target token
    x = target[0]
     # iterate over target sequence
    for i in range(1, target_len):
      # pass current input, previous hidden, cell states to Decoder LSTM
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      # get index of highest value in output tensor
      best_guess = output.argmax(1) 
      x = target[i] if random.random() < tfr else best_guess 
    return outputs


In [ ]:
# Hyperparameters
learning_rate = 0.001
step = 0
# Instantiate Seq2Seq model
model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
# Instantiate optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Set index of <pad> token
pad_idx = english.vocab.stoi["<pad>"]
# Define loss function
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

def translate_sentence(model, sentence, hindi, english, device, max_length=50):
    # If input is string
    if type(sentence) == str:
        tmp = preprocess_text_hi(sentence)
        tokens = [token.lower() for token in tmp]
    else:
        # If input is tokenized,
        tokens = [token.lower() for token in sentence]

    # Add start and end of sequence tokens
    tokens.insert(0, hindi.init_token)
    tokens.append(hindi.eos_token)

    # Convert token sequence to a tensor of indices
    text_to_indices = [hindi.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Pass input sentence through encoder LSTM
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

   # <sos> token
    outputs = [english.vocab.stoi["<sos>"]]

    # Loop until maximum output length is reached or till <eos> token
    for _ in range(max_length):
      # previous predicted word as input to decoder LSTM
        previous_word = torch.LongTensor([outputs[-1]]).to(device)
        # input word and previous hidden and cell states through decoder LSTM
        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            # index of the word with highest probability
            best_guess = output.argmax(1).item()
        # predicted word to output sequence
        outputs.append(best_guess)
        # <eos> token is predicted, break loop
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break
    # output sequence of indices to a sequence of tokens
    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # excluding the <sos> token
    return translated_sentence[1:]

# Training

In [ ]:
!nvidia-smi

Tue Apr 11 14:39:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    34W /  70W |   1459MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
epoch_loss = 0.0
num_epochs = 12

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.src.to(device)
    target = batch.trg.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear gradients
    optimizer.zero_grad()
    # Calculate loss 
    loss = criterion(output, target)
    # back-propagation
    loss.backward()
    # Cut gradient value if > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    # Update weights
    optimizer.step()
    step += 1
    epoch_loss += loss.item()

  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))


Epoch - 1 / 12
Translated example sentence 1: 
 ['scuttled', 'felicitous', 'influence', 'extremes', 'lashes', 'bookmarks', 'influence', 'extremes', 'bookmarks', 'desperate', 'extremes', 'desperate', 'prostitute', 'things', 'things', 'things', 'fame', 'fame', 'draught', 'consultation', 'center', 'swarming', 'swarming', 'spurred', 'uppressed', 'tus', 'seer', 'lights', 'seer', 'enabledgt', 'miasma', 'worldly', 'wing', 'knows', 'knows', 'pleasantry', 'pleasantry', 'pleasantry', 'encourage', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry', 'pleasantry']
saving

Epoch_Loss - 3.662508010864258

Epoch - 2 / 12
Translated example sentence 1: 
 ['open', 'in', 'in', '<unk>', '<eos>']
Epoch_Loss - 3.6439239978790283

Epoch - 3 / 12
Translated example sentence 1: 
 ['open', 'the', 'in', '<unk>', '<eos>']
Epoch_Loss - 2.9913182258605957

Epoch - 4 / 12
Translated example sentence 1: 
 ['open', 'in', '<unk>', '

In [ ]:
# Path to save model
model_path = "/model_12.pt"
# Save model
torch.save(model.state_dict(), file_path + model_path)


#Load Pretrained model for furthur epoch training

In [ ]:
# Define path where saved model is located
model_path = "model_40.pt"

# Load saved model
model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
model.load_state_dict(torch.load(file_path+model_path))


<All keys matched successfully>

In [ ]:
hd_st = "और अनुसर्ण करो उस सर्वोत्तम चीज़ का जो तुम्हारे रब की ओर से अवतरित हुई है, इससे पहले कि तुम पर अचानक यातना आ जाए और तुम्हें पता भी न हो। "
ans = translate_sentence(model, hd_st, hindi, english, device, max_length=50)
ans

['and',
 'follow',
 'the',
 'best',
 'of',
 'the',
 'has',
 'been',
 'sent',
 'down',
 'to',
 'you',
 'from',
 'your',
 'lord',
 'before',
 'the',
 'torment',
 'overtakes',
 'you',
 'suddenly',
 'while',
 'you',
 'are',
 'unaware',
 '<eos>']

In [ ]:
sentence = ' '.join(ans)
print(sentence[:-6])

and follow the best of the has been sent down to you from your lord before the torment overtakes you suddenly while you are unaware


##Run model for further epoch

In [ ]:
# Saved Model epoch
prev_epoch = 40
# New epoch
epoch_loss = 0.0
num_epochs = 10

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1+prev_epoch, num_epochs+prev_epoch))
  model.eval()
  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.src.to(device)
    target = batch.trg.to(device)
    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear gradients
    optimizer.zero_grad()
    # Calculate loss 
    loss = criterion(output, target)
    # back-propagation
    loss.backward()
    # Cut gradient value if > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    # Update weights
    optimizer.step()
    step += 1
    epoch_loss += loss.item()

  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))


Epoch - 41 / 50
Translated example sentence 1: 
 ['open', 'the', 'in', 'session', 'in', '<eos>']
saving

Epoch_Loss - 0.6145027875900269

Epoch - 42 / 50
Translated example sentence 1: 
 ['run', 'the', 'in', 'irresistible', 'session', '<eos>']
Epoch_Loss - 0.7174165844917297

Epoch - 43 / 50
Translated example sentence 1: 
 ['open', 'the', 'upper', 'find', 'conflict', 'in', 'a', 'running', 'browser', '<eos>']
Epoch_Loss - 2.4101617336273193

Epoch - 44 / 50
Translated example sentence 1: 
 ['open', 'the', 'upper', 'link', 'in', 'a', 'window', '<eos>']
Epoch_Loss - 2.1067566871643066

Epoch - 45 / 50
Translated example sentence 1: 
 ['open', 'the', 'in', 'session', 'in', 'tabs', '<eos>']
Epoch_Loss - 3.730891466140747

Epoch - 46 / 50
Translated example sentence 1: 
 ['open', 'the', 'last', 'session', 'in', 'the', 'mode', '<eos>']
Epoch_Loss - 1.1555683612823486

Epoch - 47 / 50
Translated example sentence 1: 
 ['open', '<unk>', 'in', 'in', 'a', '<eos>']
Epoch_Loss - 1.1219122409820557


###Again Check accuracy after retraining

Hindi_sentence = "और अनुसर्ण करो उस सर्वोत्तम चीज़ का जो तुम्हारे रब की ओर से अवतरित हुई है, इससे पहले कि तुम पर अचानक यातना आ जाए और तुम्हें पता भी न हो। "

Prev epoch output(Eng sentence): "and follow the best of what has been sent down to you from your lord and your lord is not negligent of you"

In [ ]:
hd_st = "और अनुसर्ण करो उस सर्वोत्तम चीज़ का जो तुम्हारे रब की ओर से अवतरित हुई है, इससे पहले कि तुम पर अचानक यातना आ जाए और तुम्हें पता भी न हो। "
ans = translate_sentence(model, hd_st, hindi, english, device, max_length=50)
ans

In [ ]:
sentence = ' '.join(ans)
print(sentence[:-6])

If accuracy is great Save model

In [ ]:
# Define path to save model
model_path = "/model_50.pt"

# Save model
torch.save(model.state_dict(), file_path + model_path)


#Save Test Output

In [ ]:
#/content/drive/MyDrive/CS779_NLP_COMP/1_MT/input_data/eng_Hindi_data_dev_X.csv
df_test = pd.read_csv(file_path+"eng_Hindi_data_dev_X.csv", header = None)
df_test.head()

In [ ]:
df_test.columns = ['Hindi']
df_test.head()

In [ ]:
tot = len(df_test['Hindi'])

In [ ]:
ans = translate_sentence(model, df_test['Hindi'][0], hindi, english, device, max_length=50)

In [ ]:
sentence = ' '.join(ans)
print(sentence[:-6])

In [ ]:
trans_test = []
cnt = 0
for i in range(tot):
  cnt = cnt + 1
  ans = translate_sentence(model, df_test['Hindi'][i], hindi, english, device, max_length=50)
  sentence = ' '.join(ans)
  # print(sentence[:-6])
  trans_test.append(sentence[:-6])
  print(cnt)


Save with string

In [ ]:
y_dev_pred_str = '\n'.join(map(str, [pred for pred in trans_test]))

with open('answer.txt', 'w', encoding='utf-8') as file:
    file.write(y_dev_pred_str)

In [ ]:
y_dev_pred_str[:1000]

Save with List

In [ ]:
len(trans_test)

40000

In [ ]:
trans_test[:4]

['and follow the best of what we have sent down to you from your lord before you suddenly it comes on you suddenly while you perceive not',
 'and thereafter a caravan of his people and he water to drawer and he he and his bucket and his to his and luck and he had given him and he merchandise which he had been given to him and he is an enemy to him',
 'whoever brings a good deed will receive better than itand and worth and such will be safe from him demand',
 '   square <unk>']

In [ ]:
with open('my_list.txt', 'w', encoding='utf-8') as file:
    for item in trans_test:
        file.write("%s\n" % item)


# References

https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
<br>
https://drive.google.com/file/d/1D8dZAC3QAtAuPKfSb0qmBm1NxPcv4vUq/view
<br>
https://torchtext.readthedocs.io/en/latest/data.html